
<style>
    @import url('https://fonts.googleapis.com/css2?family=Montserrat:wght@300&display=swap');

    body, div, h1, h2, h3, h4, h5, h6, p, span, a, li, td, th {
        font-family: 'Montserrat', sans-serif;
        font-weight: 300;
    }
</style>    
<div id='6-manejo-de-errores-y-excepciones'></div>


<style>
    @import url('https://fonts.googleapis.com/css2?family=Montserrat:wght@300&display=swap');

    body, div, h1, h2, h3, h4, h5, h6, p, span, a, li, td, th {
        font-family: 'Montserrat', sans-serif;
        font-weight: 300;
    }
</style>    
<div id='5-pantallazos-y-resultados'></div>

# Manejo de Errores en el Simulador de Trading

## Descripción General
Este documento describe los mecanismos avanzados de manejo de errores implementados en el simulador de trading. Cada componente, como `TradingEngine`, `SignalGenerator`, `Backtester` y `CryptoDataProcessor`, incluye validaciones y gestión de excepciones para garantizar la resiliencia durante la ejecución.

---

## Ejemplos de Manejo de Errores por Componente

### 1. **Clase Config**

#### Descripción
La clase `Config` valida los parámetros críticos de configuración, como el intervalo (`interval`), y lanza errores si los valores son inválidos.

#### Ejemplo
```python
class Config:
    def __init__(self, pair: str, interval: int = 1440, oversold: int = 30, overbought: int = 70):
        allowed_intervals = [1, 5, 15, 30, 60, 240, 1440, 10080, 21600]
        if interval not in allowed_intervals:
            raise ValueError(f"Intervalo inválido: {interval}. Valores permitidos: {allowed_intervals}")
        self.pair = pair
        self.interval = interval
        self.oversold = oversold
        self.overbought = overbought
```

#### Errores Lanzados
- **`ValueError`**: Si el intervalo no está en la lista de valores válidos.

---

### 2. **CryptoDataProcessor**

#### Descripción
Este componente se conecta a la API de Kraken y procesa datos. Maneja errores relacionados con la conectividad, la validación de respuestas y la integridad de los datos.

#### Ejemplo
```python
def fetch_ohlc_data(self, pair: str, interval: int, since: int = None) -> pd.DataFrame:
    try:
        response = requests.get(self.OHLC_URL, params={'pair': pair, 'interval': interval, 'since': since})
        response.raise_for_status()  # Lanza HTTPError si el código de estado es 4xx/5xx
        data = response.json()
        
        if 'error' in data and data['error']:
            raise ValueError(f"Error de API: {data['error']}")
        
        if 'result' not in data or not data['result']:
            raise ValueError("Falta el campo 'result' en la respuesta de la API.")
        
        return self._parse_ohlc_data(data)
    
    except requests.RequestException as e:
        raise ConnectionError(f"Error en la conexión con la API: {e}")
    except ValueError as e:
        raise ValueError(f"Error procesando los datos de la API: {e}")
```

#### Errores Lanzados
- **`ConnectionError`**: Problemas de red o API.
- **`ValueError`**: Campos ausentes o inválidos en la respuesta.

---

### 3. **SignalGenerator**

#### Descripción
Genera señales de trading utilizando indicadores técnicos como las Bandas de Bollinger y el RSI. Maneja datos faltantes o inválidos.

#### Ejemplo
```python
def generate_signals(self) -> pd.DataFrame:
    try:
        for i in self.data.index:
            if pd.isna(self.data.loc[i, 'close']):
                raise ValueError(f"Falta el precio de cierre en el índice {i}")
            # Lógica para generar señales...
    except ValueError as e:
        raise RuntimeError(f"Error al generar señales: {e}")
```

#### Errores Lanzados
- **`ValueError`**: Si las columnas requeridas contienen datos faltantes.
- **`RuntimeError`**: Encapsula errores menores para depuración.

---

### 4. **Backtester**

#### Descripción
Ejecuta el backtesting para evaluar el rendimiento de las señales de trading. Valida entradas como señales y capital inicial.

#### Ejemplo
```python
def run_backtest(self) -> pd.DataFrame:
    try:
        if self.data.empty:
            raise ValueError("No hay datos disponibles para el backtesting.")
        # Lógica de backtesting...
    except ValueError as e:
        raise RuntimeError(f"Error en el backtesting: {e}")
```

#### Errores Lanzados
- **`ValueError`**: Si los datos de entrada están vacíos o son inválidos.
- **`RuntimeError`**: Maneja errores que interrumpen la ejecución.

---

### 5. **TradingEngine**

#### Descripción
El `TradingEngine` coordina todos los componentes, asegurando que cada paso del proceso sea validado y esté libre de errores.

#### Ejemplo
```python
def run(self) -> None:
    try:
        self.process_data()
        self.generate_signals()
        self.backtest()
    except Exception as e:
        raise RuntimeError(f"Error en la ejecución del TradingEngine: {e}")
```

#### Errores Lanzados
- **`RuntimeError`**: Captura errores de cualquier etapa del proceso (procesamiento de datos, generación de señales, backtesting).

---

## Resumen

| Componente          | Tipo de Error       | Condición del Error                   |
|---------------------|---------------------|---------------------------------------|
| Config              | `ValueError`       | Intervalo inválido                    |
| CryptoDataProcessor | `ConnectionError`  | Fallo en la conexión con la API       |
| SignalGenerator     | `RuntimeError`     | Falta de columnas requeridas          |
| Backtester          | `ValueError`       | Datos de entrada vacíos               |
| TradingEngine       | `RuntimeError`     | Error en cualquier etapa del proceso  |

Implementar estas estrategias robustas de manejo de errores asegura una degradación controlada, una mayor capacidad de depuración y operaciones confiables.
